In [93]:
import numpy as np

In [94]:
def simulate_gbm(s_0,mu,sigma,n_sims,T,N,randomseed =3):
    
    np.random.seed(randomseed)
    
    dt = T/N
    dW = np.random.normal(scale = np.sqrt(dt),size = (n_sims,N))
    W = np.cumsum(dW, axis = 1)
    
    time_step = np.linspace(dt,T,N)
    time_steps = np.broadcast_to(time_step,(n_sims,N))
    S_t = (s_0 * np.exp((mu - 0.5 * sigma ** 2)*time_steps +sigma *W))
    S_t = np.insert(S_t, 0, s_0, axis=1)
    
    return  S_t

In [95]:
S_0 = 55
K = 60
BARRIER = 65
r = 0.06
sigma = 0.2
T = 1
N = 252
dt = T / N
N_SIMS = 10 ** 5
OPTION_TYPE = "call"
discount_factor = np.exp(-r * T)

In [96]:
gbm_sims = simulate_gbm(s_0=S_0, mu=r, sigma=sigma, 
                        n_sims=N_SIMS, T=T, N=N)

In [97]:
payoff_matrix = np.maximum(gbm_sims[:, -1] - K, 0)
payoff_matrix

array([2.29191325, 0.        , 3.55888266, ..., 2.7463855 , 0.        ,
       1.32302539])

In [98]:
payoff_matrix

array([2.29191325, 0.        , 3.55888266, ..., 2.7463855 , 0.        ,
       1.32302539])

In [99]:
maxperpath = np.max(gbm_sims,axis = 1)
maxperpath

array([64.16104146, 64.25343095, 71.95776838, ..., 65.57249044,
       72.2703556 , 70.00478594])

In [100]:
payoff_matrix = np.where(
        maxperpath > BARRIER,
        payoff_matrix,
        0)

In [101]:
premium = discount_factor * np.mean(payoff_matrix)
premium

3.6720645143741497

In [112]:
def Barrieroption(s_0,mu,sigma,n_sims,T,N,OptionType,K,BARRIER,BarrierType,randomseed =3):
    dt = T / N
    discount_factor = np.exp(-r * T)
    
    ### use geometric brownian motion to get the share price simulation data
    gbm_sims = simulate_gbm(s_0=S_0, mu=r, sigma=sigma, n_sims=N_SIMS, T=T, N=N)
    
    
    ###assume no barrier european option final payout
    if OptionType == "call":
        payoff_matrix = np.maximum(gbm_sims[:, -1] - K, 0)
    elif OptionType =="put":
        payoff_matrix = np.maximum(0,K - gbm_sims[:, -1])
    else:
         raise ValueError("unknow type")
    
    ###get option final payoff
    if BarrierType == "UO":
        maxperpath = np.max(gbm_sims,axis = 1)
        payoff_matrix = np.where(
            maxperpath > BARRIER,
            payoff_matrix,
            0)
    elif BarrierType == "UI":
        maxperpath = np.max(gbm_sims,axis = 1)
        payoff_matrix = np.where(
            maxperpath > BARRIER,
            0,
            payoff_matrix)
    elif BarrierType == "DI":
        minperpath = np.min(gbm_sims,axis = 1)
        payoff_matrix = np.where(
            minperpath > BARRIER,
            0,
            payoff_matrix)
    elif BarrierType == "DO":
        minperpath = np.min(gbm_sims,axis = 1)
        payoff_matrix = np.where(
            minperpath > BARRIER,
            payoff_matrix,
            0)
    else:
         raise ValueError("unknow type")
    
    premium = discount_factor * np.mean(payoff_matrix)
    return(premium)

In [113]:
Barrieroption(55,0.06,0.2,50,1,252,"call",60,65,"UO")

3.6720645143741497

QuantLib

In [116]:
import QuantLib as ql

In [141]:
#calendar = ql.UnitedStates()
day_counter = ql.Actual365Fixed()

In [142]:
valuation_date = ql.Date(1, 1, 2020)
expiry_date =  ql.Date(1, 1, 2021)
ql.Settings.instance().evaluationDate = valuation_date

In [143]:
T = day_counter.yearFraction(valuation_date, expiry_date)
print(f'Time to expiry in years: {T}') 

Time to expiry in years: 1.0027397260273974


In [145]:
if OPTION_TYPE == "call":
    option_type_ql = ql.Option.Call
elif OPTION_TYPE == "put":
    option_type_ql = ql.Option.Put
        
exercise = ql.EuropeanExercise(expiry_date)
payoff = ql.PlainVanillaPayoff(option_type_ql, K)

u = ql.SimpleQuote(S_0)
r = ql.SimpleQuote(r)
sigma = ql.SimpleQuote(sigma)

underlying = ql.QuoteHandle(u)
volatility = ql.BlackConstantVol(0, ql.TARGET(), 
                                 ql.QuoteHandle(sigma), 
                                 day_counter)
risk_free_rate = ql.FlatForward(0, ql.TARGET(), 
                                ql.QuoteHandle(r), 
                                day_counter)

bs_process = ql.BlackScholesProcess(
    underlying,
    ql.YieldTermStructureHandle(risk_free_rate),
    ql.BlackVolTermStructureHandle(volatility),
)

engine = ql.MCBarrierEngine(bs_process, 'PseudoRandom', timeSteps=N, 
                            seed=42, 
                            requiredSamples=N_SIMS)

option = ql.BarrierOption(ql.Barrier.UpIn, BARRIER, rebate=0, 
                          payoff=payoff, exercise=exercise)
option.setPricingEngine(engine)

option_premium_ql = option.NPV()
option_premium_ql

0.01623786784466653